##### Imports

In [1]:
import os
# TENSORFLOW LOGS:
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import os
import math
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras.api import keras

from IPython.display import clear_output

from hpcscripts.sharedutils.trainingutils import LoadModel, SetLowTFVerbose, MakeSinglePrediction, CreateWindowGenerator, GetFileList, TrainModel
from hpcscripts.sharedutils.nomalization import DF_Nomalize, denorm
from hpcscripts.sharedutils.modelutils import SelectModelPrompt
from hpcscripts.trainers.anntrainer import CreateANNModel, ImportCombinedTrainingData
from hpcscripts.trainers.modeldefinitions import  MODEL_DEFINITIONS
from hpcscripts.trainers import modeldefinitions as mdef
from hpcscripts.trainers import anntrainer
from hpcscripts.option import pathhandler as ph
from hpcscripts.option import globalparams as G_PARAMS

# Codes

#### Functions Defninition

In [3]:
def create_binary(digits_num: int, init_val: int=0):
    return [init_val for i in range (digits_num)]

def increment_binary(binary: list):
    hold = 0
    for i in range (len(binary)):
        index = len(binary) - i - 1
        
        if index == 0 and binary[index] == 1 and hold > 0:
            return -1
        if i == 0:
            binary [index] += 1

        binary [index] += hold
        hold = 0

        if binary[index] > 1:
            binary[index] = 0
            hold = 1
    
    return binary

def create_possibility_mask(digits_num: int):
    possibilities = []
    binary = create_binary(digits_num)

    while True:

        binary = increment_binary(binary)

        if binary == -1:
            break

        possibilities.append(binary.copy())

    return possibilities

def create_possible_features(feature_list: list):
    digits_num = len (feature_list)
    possi_mask = create_possibility_mask(digits_num)

    feature_possibs = []
    for possi in possi_mask:
        new_features = []
        for mask, feature in zip(possi, feature_list):
            if mask == 1:
                new_features.append(feature)

        feature_possibs.append(new_features)
    
    return feature_possibs, possi_mask

In [4]:
def print_list(the_list: list):
    for element in the_list:
        print (element)
        
def bin_to_index(binary: str = "1010"):
    index = -1
    for n, letter in enumerate(binary[::-1]):
        if letter == "1":
            index += 2**n
    return index

#### Create Possible Labels Combinations

In [5]:
feature_list = ["hralt_m", "theta_rad", "aoac_rad", "cas_mps", "gs_dev_ddm", "hdot_1_mps"]
# label_list = ["hralt_m", "theta_rad", "aoac_rad", "cas_mps"]
# feature_list = ["hralt_m", "theta_rad"]

feature_possibs, possi_mask = create_possible_features(feature_list)

print ("Num. of possibilities: {}".format(len(feature_possibs)))
print_list(feature_possibs)

Num. of possibilities: 63
['hdot_1_mps']
['gs_dev_ddm']
['gs_dev_ddm', 'hdot_1_mps']
['cas_mps']
['cas_mps', 'hdot_1_mps']
['cas_mps', 'gs_dev_ddm']
['cas_mps', 'gs_dev_ddm', 'hdot_1_mps']
['aoac_rad']
['aoac_rad', 'hdot_1_mps']
['aoac_rad', 'gs_dev_ddm']
['aoac_rad', 'gs_dev_ddm', 'hdot_1_mps']
['aoac_rad', 'cas_mps']
['aoac_rad', 'cas_mps', 'hdot_1_mps']
['aoac_rad', 'cas_mps', 'gs_dev_ddm']
['aoac_rad', 'cas_mps', 'gs_dev_ddm', 'hdot_1_mps']
['theta_rad']
['theta_rad', 'hdot_1_mps']
['theta_rad', 'gs_dev_ddm']
['theta_rad', 'gs_dev_ddm', 'hdot_1_mps']
['theta_rad', 'cas_mps']
['theta_rad', 'cas_mps', 'hdot_1_mps']
['theta_rad', 'cas_mps', 'gs_dev_ddm']
['theta_rad', 'cas_mps', 'gs_dev_ddm', 'hdot_1_mps']
['theta_rad', 'aoac_rad']
['theta_rad', 'aoac_rad', 'hdot_1_mps']
['theta_rad', 'aoac_rad', 'gs_dev_ddm']
['theta_rad', 'aoac_rad', 'gs_dev_ddm', 'hdot_1_mps']
['theta_rad', 'aoac_rad', 'cas_mps']
['theta_rad', 'aoac_rad', 'cas_mps', 'hdot_1_mps']
['theta_rad', 'aoac_rad', 'cas_mps'

In [6]:
feature_possibs[bin_to_index("1011")]

['aoac_rad', 'gs_dev_ddm', 'hdot_1_mps']

### Search for the best features

#### Functions Definition

In [7]:
def search_features(model_id: str, itter_times:int, collected_list:list):
    for features in feature_possibs:
        val_data = {}
        val_data_avg = {}

        feature_summary = {}

        G_PARAMS.FEATURE_COLUMNS = features
        for i in range (itter_times):
            clear_output(wait=True)
            print ("i = {}, feature -> {}".format(i, features))
            model, history = anntrainer.run(
                model_id,
                save_model = False,
                return_model = True
            )

            print ()

            for key in history.history.keys():
                if not key.startswith('val'):
                    continue
                min_value = min (history.history[key])
                min_epoch = history.history[key].index(min_value)
                # print ("Minimum of {:<26} ->  {} \t on epoch ->  {}".format(key, min_value, min_index)) 

                val_data[key] = val_data.get(key, [])
                val_data[key].append(min_value)

                # Min Index
                val_data[key + '_epoch'] = val_data.get(key + '_epoch', [])
                val_data[key + '_epoch'].append(min_epoch)
                

        for key in val_data.keys():
            val_data_avg[key] = np.average (val_data[key])

        feature_summary['features']     = features
        feature_summary['performance']  = val_data_avg

        collected_list.append(feature_summary)
        
        # Save collected list
        with open("features_search.pkl", 'wb') as f:
            pickle.dump(collected_list, f)


    # Save collected list
    with open("features_search.pkl", 'wb') as f:
        pickle.dump(collected_list, f)
    print ("picke saved")


#### Codes

In [8]:
model_id = 'simp_dense'
itter_times = 4
collected_list = []

if False: 
    search_features

In [9]:
with open("features_search.pkl", 'rb') as f:
    loaded_result = pickle.load(f)

loaded_result

[{'features': ['hdot_1_mps'],
  'performance': {'val_loss': 1.3101813197135925,
   'val_loss_epoch': 3.75,
   'val_mean_squared_error': 1.3101813197135925,
   'val_mean_squared_error_epoch': 3.75,
   'val_mean_absolute_error': 0.7732463926076889,
   'val_mean_absolute_error_epoch': 3.75}},
 {'features': ['gs_dev_ddm'],
  'performance': {'val_loss': 1.3930539786815643,
   'val_loss_epoch': 4.5,
   'val_mean_squared_error': 1.3930539786815643,
   'val_mean_squared_error_epoch': 4.5,
   'val_mean_absolute_error': 0.8464909940958023,
   'val_mean_absolute_error_epoch': 5.75}},
 {'features': ['gs_dev_ddm', 'hdot_1_mps'],
  'performance': {'val_loss': 1.2845712006092072,
   'val_loss_epoch': 8.5,
   'val_mean_squared_error': 1.2845712006092072,
   'val_mean_squared_error_epoch': 8.5,
   'val_mean_absolute_error': 0.7561376243829727,
   'val_mean_absolute_error_epoch': 9.5}},
 {'features': ['cas_mps'],
  'performance': {'val_loss': 1.0743668675422668,
   'val_loss_epoch': 5.5,
   'val_mean_sq

In [10]:
len (loaded_result)

63

In [11]:
sorted_features = sorted(loaded_result, key=lambda x: x['performance']['val_mean_absolute_error'])

In [26]:
best_10s = sorted_features[:20]

In [27]:
feat_counts = {}

for data in best_10s:
    for feature in data['features']:
        feat_counts[feature] = feat_counts.get(feature, 0) + 1

feat_counts

{'hralt_m': 14,
 'theta_rad': 14,
 'cas_mps': 20,
 'gs_dev_ddm': 10,
 'hdot_1_mps': 12,
 'aoac_rad': 12}

In [28]:
best_10s

[{'features': ['hralt_m', 'theta_rad', 'cas_mps', 'gs_dev_ddm', 'hdot_1_mps'],
  'performance': {'val_loss': 1.0466221570968628,
   'val_loss_epoch': 6.75,
   'val_mean_squared_error': 1.0466221570968628,
   'val_mean_squared_error_epoch': 6.75,
   'val_mean_absolute_error': 0.5957683473825455,
   'val_mean_absolute_error_epoch': 8.25}},
 {'features': ['hralt_m', 'theta_rad', 'aoac_rad', 'cas_mps', 'gs_dev_ddm'],
  'performance': {'val_loss': 1.0554557740688324,
   'val_loss_epoch': 6.75,
   'val_mean_squared_error': 1.0554557740688324,
   'val_mean_squared_error_epoch': 6.75,
   'val_mean_absolute_error': 0.5971895158290863,
   'val_mean_absolute_error_epoch': 7.75}},
 {'features': ['hralt_m', 'theta_rad', 'cas_mps'],
  'performance': {'val_loss': 1.0416475534439087,
   'val_loss_epoch': 7.5,
   'val_mean_squared_error': 1.0416475534439087,
   'val_mean_squared_error_epoch': 7.5,
   'val_mean_absolute_error': 0.5972976237535477,
   'val_mean_absolute_error_epoch': 8.25}},
 {'features'